<a href="https://colab.research.google.com/github/thuongvan23/XAI-LandCover/blob/kienNB/eurosat_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Mount Google Drive và cài đặt thư viện
from google.colab import drive
drive.mount('/content/drive')

!pip install -q kaggle tifffile rasterio torch torchvision scikit-learn pandas numpy


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.6 MB/s eta 0:00:00


In [ ]:

# 2. Cấu hình kaggle.json để tải dataset
!mkdir -p ~/.kaggle
!cp "/content/drive/MyDrive/Colab Notebooks/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:

# 3. Tải và giải nén dataset EuroSat
!kaggle datasets download -d apollo2506/eurosat-dataset
!unzip -q eurosat-dataset.zip -d ./eurosat_data


Dataset URL: https://www.kaggle.com/datasets/apollo2506/eurosat-dataset
License(s): CC0-1.0
 99% 2.02G/2.04G [00:18<00:01, 16.9MB/s]
100% 2.04G/2.04G [00:18<00:00, 120MB/s] 


In [ ]:

# 4. Kiểm tra cấu trúc thư mục và một vài file
import os
root_dir = "/content/eurosat_data/EuroSATallBands"
for root, dirs, files in os.walk(root_dir):
    level = root.replace(root_dir, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for f in files[:5]:
        print(f"{subindent}{f}")


EuroSATallBands/
  test.csv
  validation.csv
  label_map.json
  train.csv
  Industrial/
    Industrial_2499.tif
    Industrial_1893.tif
    Industrial_327.tif
    Industrial_928.tif
    Industrial_58.tif
  Residential/
    Residential_2678.tif
    Residential_2937.tif
    Residential_1563.tif
    Residential_2032.tif
    Residential_1486.tif
  AnnualCrop/
    AnnualCrop_1606.tif
    AnnualCrop_325.tif
    AnnualCrop_1185.tif
    AnnualCrop_2415.tif
    AnnualCrop_266.tif
  SeaLake/
    SeaLake_1971.tif
    SeaLake_2675.tif
    SeaLake_2896.tif
    SeaLake_2215.tif
    SeaLake_1093.tif
  Pasture/
    Pasture_1368.tif
    Pasture_209.tif
    Pasture_1953.tif
    Pasture_1732.tif
    Pasture_1415.tif
  River/
    River_1206.tif
    River_2370.tif
    River_274.tif
    River_2466.tif
    River_624.tif
  PermanentCrop/
    PermanentCrop_2412.tif
    PermanentCrop_1797.tif
    PermanentCrop_31.tif
    PermanentCrop_2158.tif
    PermanentCrop_562.tif
  HerbaceousVegetation/
    HerbaceousVege

In [ ]:

# 5. Dataset class đọc ảnh .tif 13 channels và csv có label
import pandas as pd
import numpy as np
import rasterio
import torch
from torch.utils.data import Dataset

class EuroSATAllBands(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.root_dir, row['Filename'])
        label = int(row['Label'])

        with rasterio.open(img_path) as src:
            img = src.read().astype(np.float32)  # (13, H, W)
            img /= 10000.0  # Normalize

        if self.transform:
            img = self.transform(img)
        return img, label


In [ ]:

# 6. Chuyển ảnh numpy sang tensor
def to_tensor_13(img):
    return torch.tensor(img, dtype=torch.float32)


In [ ]:

# 7. ResNet18 sửa đầu vào 13 channels, loại bỏ fully connected cuối
import torch.nn as nn
from torchvision.models import resnet18

def get_resnet13():
    model = resnet18(pretrained=False)
    model.conv1 = nn.Conv2d(13, 64, kernel_size=7, stride=2, padding=3, bias=False)
    model.fc = nn.Identity()
    return model


In [ ]:

# 8. Hàm trích xuất đặc trưng bằng model
import numpy as np

def extract_features(model, dataloader, device):
    model.to(device)
    model.eval()
    features, labels = [], []

    with torch.no_grad():
        for imgs, lbls in dataloader:
            imgs = imgs.to(device)
            feats = model(imgs).cpu().numpy()
            features.append(feats)
            labels.append(lbls.numpy())

    return np.vstack(features), np.concatenate(labels)


In [ ]:

# 9. Chuẩn bị DataLoader cho train/val/test
from torch.utils.data import DataLoader

base_path = "/content/eurosat_data/EuroSATallBands"

train_dataset = EuroSATAllBands(csv_file=os.path.join(base_path, "train.csv"),
                               root_dir=base_path,
                               transform=to_tensor_13)

val_dataset = EuroSATAllBands(csv_file=os.path.join(base_path, "validation.csv"),
                             root_dir=base_path,
                             transform=to_tensor_13)

test_dataset = EuroSATAllBands(csv_file=os.path.join(base_path, "test.csv"),
                              root_dir=base_path,
                              transform=to_tensor_13)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:

# 10. Trích xuất đặc trưng và huấn luyện SVM
import torch
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet = get_resnet13()

print("🔍 Extracting features...")
X_train, y_train = extract_features(resnet, train_loader, device)
X_val, y_val = extract_features(resnet, val_loader, device)
X_test, y_test = extract_features(resnet, test_loader, device)

print("🎓 Training SVM...")
svm = SVC(kernel='rbf', C=10, gamma='scale')
svm.fit(X_train, y_train)

train_acc = accuracy_score(y_train, svm.predict(X_train))
val_acc = accuracy_score(y_val, svm.predict(X_val))
test_acc = accuracy_score(y_test, svm.predict(X_test))

print(f"✅ Train Accuracy: {train_acc:.4f}")
print(f"✅ Validation Accuracy: {val_acc:.4f}")
print(f"✅ Test Accuracy: {test_acc:.4f}")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


🔍 Extracting features...
🎓 Training SVM...
✅ Train Accuracy: 0.8376
✅ Validation Accuracy: 0.8123
✅ Test Accuracy: 0.8249
